In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [40]:
# Load data based on user input size
n = int(input("Enter the size of your data : "))
if n == 20:
    folds = [2, 3, 5, 10]
    print("results of data 20")
    data = pd.read_csv('/Users/gunnviryasiri/Documents/CDTI4/ml/myowncode/Data/HeightWeight20.csv')
elif n == 100:
    folds = [2, 3, 5, 10, 20]
    print("results of data 100")
    data = pd.read_csv('/Users/gunnviryasiri/Documents/CDTI4/ml/myowncode/Data/HeightWeight100.csv')
elif n == 10000:
    folds = [2, 3, 5, 10, 20]
    print("results of data 10000")
    data = pd.read_csv('/Users/gunnviryasiri/Documents/CDTI4/ml/myowncode/Data/HeightWeight.csv')
else:
    print("Enter the correct data size")

X = data[['Height']].values
y = data['Weight'].values

data

results of data 20


,Height,Weight
0,187.571423,109.720985
1,174.706036,73.622732
2,188.239668,96.497550
3,182.196685,99.809504
4,177.499762,93.598619
5,170.822660,69.042216
6,174.714106,83.428219
7,173.605229,76.190352
8,170.228132,79.800187
9,161.179495,70.941642


In [42]:
# ฟังก์ชันสำหรับคำนวณความเที่ยงตรง (Precision) และความแม่นยำ (Accuracy) r2 เป็นเป็นค่าที่บอก ว่าตัวแปร Xทั้งหมดสามารถอธิบายตัวแปร Yได้เท่าไร 
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return rmse, r2

array([ 4.33677799, 10.66879098,  2.89487278,  6.0813693 , 11.16027289,
        7.39957614,  6.97799984,  8.17507691,  7.66450193,  7.22463499])

In [44]:
def cross_validation(X, y, k=10, random_state=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    
    rmse_values = []
    r2_values = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        rmse, r2 = evaluate_model(y_test, y_pred)
        rmse_values.append(rmse)
        r2_values.append(r2)
    
    return np.array(rmse_values), np.array(r2_values)

In [46]:
# ทดลอง seed และ fold ตามนี้
n_seed = 100
seeds = range(1, n_seed+1)

results = {}

# วนลูปทดสอบทุก seed และ fold
for fold in folds:
    fold_results = {}
    for seed in seeds:
        rmse_cv, r2_cv = cross_validation(X, y, k=fold, random_state=seed)
        fold_results[f"seed {seed}"] = (rmse_cv, r2_cv)
    results[f"fold {fold}"] = fold_results

seeds

range(1, 101)

In [ ]:
# แสดงผล และคำนวณ sd , average
for fold, fold_result in results.items():
    print(f"{fold}:")
    rmse_list = []
    r2_list = []
    for seed, (rmse, r2) in fold_result.items():
        # print(f"  {seed} - RMSE: {float(np.mean(rmse)):.2f}\t R2: {float(np.mean(r2)):.2f}")
        rmse_list.append(rmse)
        r2_list.append(r2)
    
    # หาค่า standard deviation and average จาก list
    rmse_mean = np.mean(rmse_list)
    rmse_sd = np.std(rmse_list)
    r2_mean = np.mean(r2_list)
    r2_sd = np.std(r2_list)

    
    print(f"  Average RMSE: {float(rmse_mean):.4f}\t SD RMSE: {float(rmse_sd):.4f}")
    print(f"  Average R2: {float(r2_mean):.4f}\t SD R2: {float(r2_sd):.4f}")
    print()